# データ可視化対象の探索

## 準備

In [2]:
# warningsモジュールのインポート
import warnings

# データ解析や機械学習のライブラリ使用時の警告を非表示にする目的で警告を無視
# 本書の文脈では、可視化の学習に議論を集中させるために選択した
# ただし、学習以外の場面で、警告を無視する設定は推奨しない
warnings.filterwarnings("ignore")

In [49]:
# bs4モジュールのインポート
from bs4 import BeautifulSoup

# jsonモジュールのインポート
import json

# osモジュールのインポート
import os

# requestモジュールのインポート
import requests

# timeモジュールのインポート
import time

## API利用

`config.json`に以下の形式でapp_idが保存済みという前提。

```json
{
  "app_id": "your_application_id_here"
}

```

In [37]:
# 設定ファイルからアプリケーションIDを読み込む
try:
    with open("config.json") as f:
        config = json.load(f)
        app_id = config["app_id"]
except (FileNotFoundError, KeyError):
    raise ValueError("アプリケーションIDが設定されていません。config.jsonファイルにapp_idを設定してください。")

In [38]:
# e-Stat APIの統計データ取得エンドポイントを設定
base_url = "https://api.e-stat.go.jp/rest/3.0/app/json/getStatsData"

In [39]:
# リクエストパラメータを設定
params = {
    "appId": app_id,
    "statsDataId": "0003348423",  # 景気ウォッチャー調査の統計表ID
}

In [40]:
# APIにリクエストを送信し、レスポンスを取得
response = requests.get(base_url, params=params)

# レスポンスのJSONデータを取得
json_data = response.json()

# TABLE_INFの情報を取得
table_inf = json_data["GET_STATS_DATA"]["STATISTICAL_DATA"]["TABLE_INF"]

# 統計表情報の出力
print("統計表ID:", table_inf["@id"])
print("政府統計名:", table_inf["STAT_NAME"]["$"])
print("政府統計コード:", table_inf["STAT_NAME"]["@code"])
print("調査年月:", table_inf["SURVEY_DATE"])
print("公開日:", table_inf["OPEN_DATE"])
print("小地域属性フラグ:", table_inf["SMALL_AREA"])
print("統計表題名:", table_inf["TITLE"])

統計表ID: 0003348423
政府統計名: 景気ウォッチャー調査
政府統計コード: 00100001
調査年月: 202409
公開日: 2024-10-23
小地域属性フラグ: 0
統計表題名: 季節調整値　全国の分野・業種別ＤＩの推移


> このサービスは、政府統計総合窓口(e-Stat)のAPI機能を使用していますが、サービスの内容は国によって保証されたものではありません。

## スクレイピング・クローリング

In [50]:
# データ収集対象のURL
base_url = "https://books.toscrape.com/catalogue/page-{}.html"

In [51]:
# 試験的に1ページ目から2ページ目までのデータに限定
for page in range(1, 3):

    # 取得対象のページ数を表示
    print(f"# Page: {page}")

    # 指定したURLにGETリクエストを送信し、レスポンスを取得
    response = requests.get(base_url.format(page))
    # レスポンスのHTMLコンテンツをBeautifulSoupでパース
    soup = BeautifulSoup(response.text, "html.parser")
    # class属性が"product_pod"である<article>要素を全て取得
    books = soup.find_all("article", class_="product_pod")

    # booksの情報を一つ一つ処理
    for i, book in enumerate(books):

        # 全てを表示すると紙面を圧迫するため、5番目までに限定
        if i < 5:
            
            # <h3>要素内の<a>要素のtitle属性からタイトルを取得
            title = book.h3.a["title"]
            # class属性が"price_color"である<p>要素のテキストから価格を取得
            price = book.find("p", class_="price_color").text
            # bookのtitleとpriceを表示
            print(f"- {title}: {price}")

    # 標準出力の見栄えを整えるために改行
    print("")
    
    # サーバーへの負荷を考慮して、1ページ取得ごとに1秒待つ
    time.sleep(1)

# Page: 1
- A Light in the Attic: Â£51.77
- Tipping the Velvet: Â£53.74
- Soumission: Â£50.10
- Sharp Objects: Â£47.82
- Sapiens: A Brief History of Humankind: Â£54.23

# Page: 2
- In Her Wake: Â£12.84
- How Music Works: Â£37.32
- Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide to Making Small Batch Jams, Jellies, Pickles, Condiments, and More: Â£30.52
- Chase Me (Paris Nights #2): Â£25.27
- Black Dust: Â£34.53

